In [1]:
# automatically show plots inside the notebook
%matplotlib inline  

# reload all modules before executing code
%load_ext autoreload
%autoreload 2  

import numpy as np
import os

%cd ../src
from wavenet.wave_train import *
from dataset import TextToSpeechDataset, WavenetLoader
import hparam as utils
from wavenet.utils import wav_to_spectrogram
import wavenet.wave_model as WavenetModel
import wavenet.wave_util as util

/Users/evinpinar/Documents/Classes/IDP-Luminovo/tts/src


In [2]:
hparams = utils.load_params_from_yaml("wavenet/wave_hparams.yaml")

In [10]:
csv_file = "data/metadata.csv"
audio_dir = 'data/wavs'

In [11]:
TTSDataset = TextToSpeechDataset(csv_file, audio_dir)
print(len(TTSDataset))

13084


In [4]:
model = util.fetch_model(hparams)

In [12]:
dataloader = util.fetch_dataloader(TTSDataset, model, hparams)

sample size:  31


In [7]:
sample_size = model.receptive_field + hparams.output_length
print(sample_size)
print(model.receptive_field)
print(hparams.output_length)
print(dataloader['val'].sample_size)

16
15
1
16


In [13]:
import os
import logging
logging.basicConfig(level=logging.WARNING)
log = logging.getLogger()

_logdir = '/root/env/IDP-Luminovo/tts/log/wavenet/'
logdir = os.path.join(_logdir, "lab_test")
if not os.path.isdir(logdir):
        log.info("Creating directory {}".format(logdir))
        os.makedirs(logdir)
        os.chmod(logdir, 0o775)

In [4]:
pwd

'/root/env/IDP-Luminovo/tts'

In [36]:
checkpt = 'wavenet/checkpoints/best.pth.tar'
if not os.path.exists(checkpt):
    raise("File doesn't exist {}".format(checkpt))
state = torch.load(checkpt)
if hparams.resume:
    log.info('Resuming training from checkpoint: {}'.format(checkpt))
    best_metric = state['best_metric']
    global_step = state['global_step']
    optimizer.load_state_dict(state['optim_dict'])
log.info('Loading model from checkpoint: {}'.format(checkpt))
model.load_state_dict(state['state_dict'])

In [37]:
gen = model.generate(1000)

15

In [40]:
import IPython.display as ipd
ipd.Audio(gen, rate=22050)

In [ ]:
train_and_evaluate(TTSDataset, hparams, logdir)

sample size:  31
epoch:  0
collate call!:  5
[127133, 39069, 191901, 140445, 65949]
0
 length of audio:  127133
 max len:  191901
 batch input:  torch.Size([1, 1, 191901])
 batch output:  torch.Size([191871])
1
 length of audio:  39069
 max len:  191901
 batch input:  torch.Size([2, 1, 191901])
 batch output:  torch.Size([383742])
2
 length of audio:  191901
 max len:  191901
 batch input:  torch.Size([3, 1, 191901])
 batch output:  torch.Size([575613])
3
 length of audio:  140445
 max len:  191901
 batch input:  torch.Size([4, 1, 191901])
 batch output:  torch.Size([767484])
4
 length of audio:  65949
 max len:  191901
 batch input:  torch.Size([5, 1, 191901])
 batch output:  torch.Size([959355])
return batches
data loading 
forward step! 
input size:  torch.Size([5, 1, 191901])
  size after dilations:  torch.Size([5, 4, 191871])
  output size:  torch.Size([5, 256, 191871])


In [19]:
from src.dataset import TextToSpeechDataset, WavenetLoader
dataloader = util.fetch_dataloader(TTSDataset, model, hparams)
loader = dataloader['val']
print(loader.q_channels)

256


In [20]:
TTSDataset[0]['speech']

array([-7.3242188e-04, -7.6293945e-04, -6.4086914e-04, ...,
        7.3242188e-04,  2.1362305e-04,  6.1035156e-05], dtype=float32)

## Generation fix

In [10]:
model.receptive_field


15

In [35]:
gen = model.generate(20)
gen = torch.tensor(gen).float().view(1, 1, -1)
gen.size()

generation starting size:  torch.Size([1, 1, 15])
generation starting size:  torch.Size([1, 1, 15])
generation starting size:  torch.Size([1, 1, 15])
generation starting size:  torch.Size([1, 1, 15])
generation starting size:  torch.Size([1, 1, 15])


torch.Size([1, 1, 20])

In [23]:
next_gen = model(gen)

In [24]:
next_gen.size()

torch.Size([1, 20, 256])

In [25]:
next_gen[:, -1, :].size() 

torch.Size([1, 256])

In [29]:
_gen = next_gen[:, -1, :]
_gen.size()

torch.Size([1, 256])

In [27]:
next_gen[0, 19, 155]

tensor(-0.2211)

In [31]:
_gen[0, 155]

tensor(-0.2211)

In [32]:
k, i = _gen.max(1)

In [33]:
k

tensor([ 0.5955])

In [34]:
i

tensor([ 215])